In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.14.0
pandas version:   0.22.0
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML file. (Actually, there are several runs that have been concatinated into a YAML file that has a sequence in its top level.)

In [3]:
filename = 'miniGraphics-skybridge-vn-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.DataFrame(yaml_data)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [4]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [5]:
data = flatten_table(data)

Add a column that gives a human-readable name to each image resolution.

In [6]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data['image-size'] = data['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [7]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data['composite-algorithm'] = data['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [8]:
import IPython.display

data_description = ''

for column_name in data.columns:
    data_description = data_description + '**' + column_name + '** '
    unique_values = data[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    elif (numpy.issubdtype(unique_values.dtype, numpy.number)):
        data_description = (
            data_description +
            str(numpy.nanmin(unique_values)) + ' &ndash; ' +
            str(numpy.nanmax(unique_values)) + ' '
        )
    elif not pandas.isnull(unique_values).any():
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

**color-buffer-format** byte  
**composite-algorithm** Naive Remainder Telescoping nan 2-3 Swap IceT  
**composite-seconds** 0.00227687 &ndash; 28.7058  
**compress-seconds**  
**construct-tree-seconds** 9.1633e-05 &ndash; 0.061385  
**depth-buffer-format** float  
**gather-seconds** 0.00095674 &ndash; 4.52609  
**geometry** box  
**geometry-distribution** duplicate  
**geometry-overlap** -0.05  
**icet-copy-result-seconds**  
**image-compression** True False  
**image-height** 500 1080 4320  
**image-width** 500 1920 7680  
**k**  
**max-image-split** nan 1000000.0  
**num-processes** 64 &ndash; 8192  
**num-triangles** 768 &ndash; 98304  
**paint-seconds** 0.00020071 &ndash; 0.170633  
**painter** simple  
**partial-composite-seconds** 0.00037141 &ndash; 27.0723  
**phi-rotation** -178.406 &ndash; 136.756  
**random-seed** 17627  
**rendering-order-dependent** False  
**start-time** 2018-04-03T12:58:47.000000000 &ndash; 2018-04-18T10:04:49.000000000  
**theta-rotation** -177.945 &ndash; 179.418  
**total-seconds** 0.00280725 &ndash; 28.7062  
**trial-num** 0 &ndash; 19  
**uncompress-seconds**  
**zoom** 1.5  
**image-size** Desktop Window HDTV 8K UHD  


## Plot Data

We are plotting for the 2-3 swap algorithm the time it takes to the "partial composite" breaking it up into the time for building the compositing tree and for everything else.

In [9]:
data['transfer-blend-seconds'] = data['partial-composite-seconds'] - data['construct-tree-seconds']

In [10]:
averages = data.pivot_table(
    values=['transfer-blend-seconds', 'construct-tree-seconds'],
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

averages

construct-tree-seconds                           \
image-size                          8K UHD Desktop Window      HDTV   
composite-algorithm               2-3 Swap       2-3 Swap  2-3 Swap   
num-processes                                                         
64                                0.000125       0.000101  0.000108   
96                                0.000148       0.000131  0.000134   
128                               0.000227       0.000197  0.000220   
144                               0.000251       0.000215  0.000237   
192                               0.000275       0.000268  0.000274   
256                               0.000424       0.000402  0.000432   
288                               0.000457       0.000453  0.000470   
384                               0.000569       0.000574  0.000572   
432                               0.000678       0.000686  0.000669   
512                               0.000927       0.000919  0.000903   
528                               0.000976       0.000961  0.000941   
576                               0.001143       0.001039  0.001017   
1024                              0.002322       0.002213  0.002231   
1056                              0.002432       0.002394  0.002396   
1152                              0.002615       0.002631  0.002553   
1296                              0.002962       0.003020  0.002953   
1536                              0.003684       0.003628  0.003670   
1584                              0.003884       0.003874  0.003702   
1728                              0.004451       0.004209  0.004305   
2048                              0.006172       0.005817  0.006099   
2112                              0.006159       0.006153  0.006366   
2304                              0.006839       0.006973  0.006804   
2592                              0.008079       0.008244  0.008308   
3072                              0.010270       0.010395  0.010630   
3168                              0.010902       0.011236  0.011353   
3456                              0.012580       0.012744  0.012627   
3888                              0.015432            NaN  0.016218   
4096                              0.017546       0.017342  0.018047   
4608                              0.021259       0.020805  0.021979   
5184                              0.025702       0.025236  0.026376   
6144                              0.034067       0.033497  0.033971   
6912                              0.041714       0.041593  0.042459   
7776                              0.051699            NaN  0.051798   
8192                              0.058230       0.058189  0.059076   

                    transfer-blend-seconds                           
image-size                          8K UHD Desktop Window      HDTV  
composite-algorithm               2-3 Swap       2-3 Swap  2-3 Swap  
num-processes                                                        
64                                0.385656       0.002477  0.021402  
96                                0.386075       0.002551  0.020028  
128                               0.376058       0.002407  0.020491  
144                               0.385088       0.002495  0.020583  
192                               0.377050       0.002509  0.019934  
256                               0.373612       0.002496  0.020438  
288                               0.380985       0.002609  0.020247  
384                               0.371444       0.002506  0.019615  
432                               0.378975       0.002633  0.020586  
512                               0.373236       0.002493  0.020675  
528                               0.374137       0.002530  0.020543  
576                               0.420300       0.002527  0.020245  
1024                              0.372319       0.002674  0.020487  
1056                              0.370557       0.002953  0.020358  
1152                              0.372480       0.002796  0.

Create a filled chart showing the times to construct the composite tree and the rest of the composite.

In [11]:
image_size = 'HDTV'
algorithm = '2-3 Swap'

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[64, 128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

x = averages.index
y = numpy.column_stack((
    numpy.zeros(numpy.shape(x)),
    numpy.array(averages['transfer-blend-seconds', image_size, algorithm]),
    numpy.array(averages['construct-tree-seconds', image_size, algorithm] +
                averages['transfer-blend-seconds', image_size, algorithm]),
))

axes.fill(
    x,
    y,
    color=toyplot.color.Palette()[0],
    opacity=[1.0, 0.6],
)

axes.text(
    1300,
    0.04,
    'Constructing Composite Tree',
    color=toyplot.color.Palette()[0],
    #opacity=0.6,
)

axes.text(
    1300,
    0.01,
    'Remaining Compositing (Transfer/Blending)',
    color='white',
)

<polygon points="45.0,238.0 76.08657860975289,238.0 98.14285714285714,238.0 107.17315721950578,238.0 129.22943575261004,238.0 151.28571428571428,238.0 160.31601436236292,238.0 182.37229289546718,238.0 191.40259297211574,238.0 204.42857142857147,238.0 206.78780177162065,238.0 213.45887150522,238.0 257.5714285714286,238.0 259.9306589144778,238.0 266.6017286480772,238.0 275.6320287247258,238.0 288.6580071811814,238.0 291.0172375242307,238.0 297.68830725783005,238.0 310.7142857142857,238.0 313.073516057335,238.0 319.74458579093437,238.0 328.77488586758295,238.0 341.80086432403857,238.0 344.16009466708783,238.0 350.8311644006872,238.0 359.86146447733586,238.0 363.85714285714283,238.0 372.8874429337915,238.0 381.9177430104401,238.0 394.94372146689574,238.0 403.9740215435443,238.0 413.0043216201929,238.0 417.0,238.0 417.0,160.45167272341644 413.0043216201929,174.8317342001306 403.9740215435443,177.57805208103719 394.94372146689574,179.94004598857015 381.9177430104401,183.22699526749318 372.8874429337915,184.26222880773807 363.85714285714283,182.1101343798172 359.86146447733586,184.8510684340684 350.8311644006872,182.59005820806217 344.16009466708783,183.63429078173337 341.80086432403857,185.28948098910737 328.77488586758295,185.9901109337762 319.74458579093437,185.61937168459676 313.073516057335,186.5806038708988 310.7142857142857,185.66053333735223 297.68830725783005,184.46732469932974 291.0172375242307,185.47357503251123 288.6580071811814,187.38926120018434 275.6320287247258,187.19007110380795 266.6017286480772,187.82778290902593 259.9306589144778,187.87563495124706 257.5714285714286,187.5594057158958 213.45887150522,188.15401580314025 206.78780177162065,187.42079056331718 204.42857142857147,187.0955477680561 191.40259297211574,187.3154861146879 182.37229289546718,189.70630007307406 160.31601436236292,188.15020321674885 151.28571428571428,187.67977117338205 129.22943575261004,188.91920792408249 107.17315721950578,187.32275649657694 98.14285714285714,187.5477131277312 76.08657860975289,188.68810486692402 45.0,185.304597559712" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="45.0,185.304597559712 76.08657860975289,188.68810486692402 98.14285714285714,187.5477131277312 107.17315721950578,187.32275649657694 129.22943575261004,188.91920792408249 151.28571428571428,187.67977117338205 160.31601436236292,188.15020321674885 182.37229289546718,189.70630007307406 191.40259297211574,187.3154861146879 204.42857142857147,187.0955477680561 206.78780177162065,187.42079056331718 213.45887150522,188.15401580314025 257.5714285714286,187.5594057158958 259.9306589144778,187.87563495124706 266.6017286480772,187.82778290902593 275.6320287247258,187.19007110380795 288.6580071811814,187.38926120018434 291.0172375242307,185.47357503251123 297.68830725783005,184.46732469932974 310.7142857142857,185.66053333735223 313.073516057335,186.5806038708988 319.74458579093437,185.61937168459676 328.77488586758295,185.9901109337762 341.80086432403857,185.28948098910737 344.16009466708783,183.63429078173337 350.8311644006872,182.59005820806217 359.86146447733586,184.8510684340684 363.85714285714283,182.1101343798172 372.8874429337915,184.26222880773807 381.9177430104401,183.22699526749318 394.94372146689574,179.94004598857015 403.9740215435443,177.57805208103719 413.0043216201929,174.8317342001306 417.0,160.45167272341644 417.0,15.0 413.0043216201929,47.299279069293746 403.9740215435443,73.04019553174558 394.94372146689574,96.30009940654732 381.9177430104401,118.28659696449822 372.8874429337915,130.1475199896481 363.85714285714283,137.67664152971514 359.86146447733586,144.9194395142323 350.8311644006872,151.5021401362367 344.16009466708783,155.68299750570003 341.80086432403857,159.11624570834158 328.77488586758295,165.53408305312965 319.74458579093437,168.8668030656765 313.073516057335,170.90788479351434 310.7142857142857,170.6450621337384 297.68830725783005,173.86733983637382 291.0172375242307,176.359895643827 288.6580071811

In [12]:
toyplot.pdf.render(canvas, '2-3-swap-overhead.pdf')